In [1]:
import os
import json
from typing import Union
import requests

'''
Functions to read and write files
'''

def read_file(file_name: str) -> str:
    '''
    Read the text file and return the content.
    '''
    with open(file_name, 'r') as f:
        return f.read()


def read_json_file(file_name: str) -> Union[list, dict]:
    '''
    Read the json file and return the content.
    '''
    return json.loads(read_file(file_name))


def write_file(file_name: str, content: str):
    '''
    Write the downloaded content in text format into files.
    '''
    parent_dir = os.path.dirname(file_name)
    if not os.path.exists(parent_dir):
        # create new directory 
        os.makedirs(parent_dir, exist_ok=True)
    with open(file_name, 'w') as f:
        print(f'writing to {file_name}')
        f.write(content)


def write_json_file(file_name: str, content: Union[list, dict]):
    '''
    Write the downloaded content in json format into files.
    '''
    write_file(file_name, json.dumps(content, indent=4))



In [ ]:
from glob import glob
from typing import Union
import pandas as pd
import sqlite3
from project.util import read_file, read_json_file

'''
This project aims to explore the necessary job skills required for each job title, namely data analyst, data engineer and data scientist. 
Skills requirements appear in job details, and for analysis of significant level of those skills, we need to extract each line of the details.
Then we will conduct dimensional modelinng.
The fact refers to each job details and the fact table is composed of job id, section id, line id, job details.
The dimensional table contains all the dimensions of each job such as the job title, job company, job area, etc.
The grain of fact table are job_id, section_id, and line_id which can be used to identify each job details. 
The grain of dimensional table is job_id which could be used to identify job information.

Therefore, this python file is used for transforming the data into Job Details Fact table and Job Info Dimension Table.
'''

data = []
lines = []
for details_file in glob(f'downloads/seek/jobs/job_details_*.json'):
    details = read_json_file(details_file)
    info = details.copy()
    # data list exclude job details(description) 
    details.pop('job_lines')
    data.append(details)

    # extract line_id and section_id and each line's job description
    for sec_num, section in enumerate(info['job_lines']):
        for line_num, line in enumerate(section):
            info_dict = {}
            info_dict['job_id'] = info['job_id']
            info_dict['section_id'] = sec_num + 1
            info_dict['line_id'] = line_num + 1
            info_dict['detail'] = line
            lines.append(info_dict)

# create data frame to be transformed to sqlite
info_df = pd.DataFrame(data)
details_df = pd.DataFrame(lines)

# connect to database
con = sqlite3.connect('seek_jobs.sqlite3')
# import data frame to sqlite
info_df.to_sql('dim_job_info', con=con, index=False, if_exists='replace')
details_df.to_sql('fact_job_info', con=con, index=False, if_exists='replace')

In [ ]:
import datetime
import re
from dateutil.relativedelta import relativedelta


'''
This python file is used to transfrom datetime data into consistent data formats.

'''


def get_past_date(str_days_ago):

    
'''
The job Ad posted time usually have the following suffixes:
1. 'm' means the job ad was posted minutes ago; 
2. 'h' means the job ad was posted hours before; 
3. 'd' means the job ad was posted days ago.
Based on these suffixes, we can use the Python datetime library 
to calculate the Universal Time Coordinated date of the job ad posted time.
'''


    if str_days_ago:
        TODAY = datetime.datetime.today()
        dates = str_days_ago.split()[1]
        splitted = re.findall(r'(\d+)(\w)', dates)[0]

        if splitted[1].lower() in ['hour', 'hours', 'hr', 'hrs', 'h']:
            date = datetime.datetime.now() - relativedelta(hours=int(splitted[0]))
            return str(date.isoformat())
        elif splitted[1].lower() in ['day', 'days', 'd']:
            date = TODAY - relativedelta(days=int(splitted[0]))
            return str(date.isoformat())
        elif splitted[1].lower() in ['wk', 'wks', 'week', 'weeks', 'w']:
            date = TODAY - relativedelta(weeks=int(splitted[0]))
            return str(date.isoformat())
        elif splitted[1].lower() in ['mon', 'mons', 'month', 'months', 'm']:
            date = TODAY - relativedelta(months=int(splitted[0]))
            return str(date.isoformat())
        elif splitted[1].lower() in ['yrs', 'yr', 'years', 'year', 'y']:
            date = TODAY - relativedelta(years=int(splitted[0]))
            return str(date.isoformat())
        else:
            return "Wrong Argument format"
    else:
        return "Null"
    
    
# Read the CSV file into pandas dataframe
job_details = pd.read_csv('seek_data_analyst_job_info_cleaned.csv')
# Create a new list to contain the transformed job_posted_time
job_posted_time = []


for dates in job_details['job_listing_date']:
    job_posted_time.append(get_past_date(dates))
    

# Create new column to the job_posted_time dataframe
job_details['job_posted_time'] = job_posted_time

### Extract Data Analyst, Data Engineer, Data Scientist from More Than 10,000 Job Occupations

The search algorithm from SEEK website confuses those job classes when we conduct exact matching, such that business intelligence, accountant will be appeared in searching data analyst jobs. Therefore, we need to filter out the related recruitment data for Data Analyst, Data Engineer, Data Scientist. In this project, we use SQL LIKE and UNLIKE operator to identify whether if the certain job belongs to these three titles or not. Finally, from 16,000 job advertisements, there are 2995 job advertisements for Data Analyst, 1426 job advertisements for Data Engineer, and 729 job advertisements for Data Scientist.

``` sql
WITH 
enriched AS (
    SELECT job_id, job_title
            , CASE 
            WHEN job_title LIKE '%data%analyst%'  THEN 'data analyst'  
            WHEN job_title LIKE '%bi%'  THEN 'data analyst'  
            WHEN job_title LIKE '%tableau%'  THEN 'data analyst'
            WHEN job_title like '%sql%' THEN 'data analyst'     
            WHEN job_title LIKE '%analytic%'  THEN 'data analyst' 
            WHEN job_title LIKE '%data%modeller%'  THEN 'data analyst' 
            WHEN job_title LIKE '%visualisation%'  THEN 'data analyst' 
            WHEN job_title LIKE '%business%intelligence%'  THEN 'data analyst'  
            WHEN job_title like '%insight%' THEN 'data analyst'
            WHEN job_title like '%reporting%' THEN 'data analyst'
            WHEN job_title like '%model%' THEN 'data analyst'
            
            WHEN job_title like '%data%engineer%' THEN 'data engineer'
            WHEN job_title like '%warehouse%' THEN 'data engineer'
            WHEN job_title like '%architect%' THEN 'data engineer'
            WHEN job_title like '%snowflake%' THEN 'data engineer'
            WHEN job_title like '%etl%' THEN 'data engineer'
            WHEN job_title like '%api%' THEN 'data engineer'
            WHEN job_title like '%cloud%' THEN 'data engineer'
            WHEN job_title like '%aws%' THEN 'data engineer'
            WHEN job_title like '%kafka%' THEN 'data engineer'
            WHEN job_title like '%pipeline%' THEN 'data engineer'
            WHEN job_title like '%migration%' THEN 'data engineer'

            WHEN job_title like '%scientist%' THEN 'data scientist'
            WHEN job_title like '%science%' THEN 'data scientist'
            WHEN job_title like '%machine%learning%' THEN 'data scientist'
            WHEN job_title like '% ai%' THEN 'data scientist'
            WHEN job_title like '%ai %' THEN 'data scientist'
            WHEN job_title like '%ml%' THEN 'data scientist'
            WHEN job_title like '%computational%statistics%' then 'data scientist'
            WHEN job_title like '%quantitative%' then 'data scientist'
            WHEN job_title like '%artificial%intelligence%' then 'data scientist'

            WHEN job_title like '%business%analyst%' THEN 'else'
            WHEN job_title like '%accountant%' THEN 'else'
            WHEN job_title like '%sales%' THEN 'else'
            WHEN job_title like '%product%' THEN 'else'
            WHEN job_title like '%dev%ops%' THEN 'else'
            WHEN job_title like '%graduate%' THEN 'else'
            WHEN job_title LIKE '%developer%' THEN 'else'
            WHEN job_title LIKE '%functional%' THEN 'else'
            WHEN job_title LIKE '%financial%' THEN 'else'
            WHEN job_title LIKE '%finance%' THEN 'else'
            WHEN job_title LIKE '%manager%' THEN 'else'
            WHEN job_title LIKE '%project%' THEN 'else'
            WHEN job_title LIKE '%program%' THEN 'else'

            END AS job_standard_name
    FROM dim_job_info
    ORDER BY 2 DESC
    )

, filtered_job_details AS (
    SELECT *
    FROM dim_job_details
    WHERE job_id IN (
        SELECT job_id FROM enriched 
        WHERE job_standard_name = 'data analyst' )
--      WHERE job_standard_name = 'data engineer' )
--      WHERE job_standard_name = 'data scientist' )
    )

-- Select the job_details of each job_standard_name and transfrom them into CSV files

SELECT *
FROM filtered_job_details

-- Count the JD numbers of each job_standard_name
SELECT job_standard_name, count(1)
FROM enriched
GROUP BY 1
```

### Term Frequency Analysis and Skills Required to be a Data Analyst, Data Engineer and Data Scientist.

After we filter out the three kinnds of jobs, we can explore the key words in those jobs and extract skills from those jobs.

In [ ]:
from typing import List
import re
from typing import Iterable, Any
import pandas as pd
import nltk
from nltk.corpus import stopwords


'''
From the filtered JDs above, this python file is used to calculate the term frequency of each word and phrase. 
Then we can filter out the meaningful word and phrase with high appeared frequency including technical and soft skills.
'''

# Download the stopword from nltk library.
# nltk.download('stopwords')


words = {}


def ngram(items: Iterable[Any], n: int) -> List[Any]:
    
    '''
    Return grams list.
    '''
    
    items = list(items)
    return [items[i: i+n] for i in range(len(items) - n + 1)]


def get_words(line: str, gram: int) -> List[str]:
    # extract words
    words = filter(lambda w: len(w) >= 2 or w == 'r', re.findall(r'[a-z/]+', str(line).lower()))
    if gram == 1:
        stop_words = get_stop_words('en')
        # remove stopwords
        words = [word for word in words if word not in stop_words]
        
    words = [' '.join(w) for w in ngram(words, gram)]
    return words


def word_count(df, gram: int = 1):
    '''
    Return the calculated term frequency of each word and phrase from each line of the job_detail. 
    '''
    for line in df['detail']:
        for word in get_words(line, gram):
            if word in words:
                words[word] += 1
            else:
                words[word] = 1

df = pd.read_csv('da_details_line.csv')
# df = pd.read_csv('de_details_line.csv')
# df = pd.read_csv('ds_details_line.csv')

word_count(df, 1)
# sort words or phrases in descending order
for word, count in sorted(words.items(), key=lambda t: t[-1] * -1):
    print(word, count)

After we filter out the important skills, we can use SQL LIKE operator to identify which statements contain these keywords and count the distinct total number of job ads.

``` sql
WITH 
enriched AS (
    SELECT job_id, job_title
            , CASE 
            WHEN job_title LIKE '%data%analyst%'  THEN 'data analyst'  
            WHEN job_title LIKE '%bi%'  THEN 'data analyst'  
            WHEN job_title LIKE '%tableau%'  THEN 'data analyst'
            WHEN job_title like '%sql%' THEN 'data analyst'     
            WHEN job_title LIKE '%analytic%'  THEN 'data analyst' 
            WHEN job_title LIKE '%data%modeller%'  THEN 'data analyst' 
            WHEN job_title LIKE '%visualisation%'  THEN 'data analyst' 
            WHEN job_title LIKE '%business%intelligence%'  THEN 'data analyst'  
            WHEN job_title like '%insight%' THEN 'data analyst'
            WHEN job_title like '%reporting%' THEN 'data analyst'
            WHEN job_title like '%model%' THEN 'data analyst'

            WHEN job_title like '%data%engineer%' THEN 'data engineer'
            WHEN job_title like '%warehouse%' THEN 'data engineer'
            WHEN job_title like '%architect%' THEN 'data engineer'
            WHEN job_title like '%snowflake%' THEN 'data engineer'
            WHEN job_title like '%etl%' THEN 'data engineer'
            WHEN job_title like '%api%' THEN 'data engineer'
            WHEN job_title like '%cloud%' THEN 'data engineer'
            WHEN job_title like '%aws%' THEN 'data engineer'
            WHEN job_title like '%kafka%' THEN 'data engineer'
            WHEN job_title like '%pipeline%' THEN 'data engineer'
            WHEN job_title like '%migration%' THEN 'data engineer'

            WHEN job_title like '%scientist%' THEN 'data scientist'
            WHEN job_title like '%science%' THEN 'data scientist'
            WHEN job_title like '%machine%learning%' THEN 'data scientist'
            WHEN job_title like '% ai%' THEN 'data scientist'
            WHEN job_title like '%ai %' THEN 'data scientist'
            WHEN job_title like '%ml%' THEN 'data scientist'
            WHEN job_title like '%computational%statistics%' then 'data scientist'
            WHEN job_title like '%quantitative%' then 'data scientist'
            WHEN job_title like '%artificial%intelligence%' then 'data scientist'

            WHEN job_title like '%business%analyst%' THEN 'else'
            WHEN job_title like '%accountant%' THEN 'else'
            WHEN job_title like '%sales%' THEN 'else'
            WHEN job_title like '%product%' THEN 'else'
            WHEN job_title like '%dev%ops%' THEN 'else'
            WHEN job_title like '%graduate%' THEN 'else'
            WHEN job_title LIKE '%developer%' THEN 'else'
            WHEN job_title LIKE '%functional%' THEN 'else'
            WHEN job_title LIKE '%financial%' THEN 'else'
            WHEN job_title LIKE '%finance%' THEN 'else'
            WHEN job_title LIKE '%manager%' THEN 'else'
            WHEN job_title LIKE '%project%' THEN 'else'
            WHEN job_title LIKE '%program%' THEN 'else'

            END AS job_standard_name
    FROM dim_job_info
    ORDER BY 2 DESC
    )

, filtered_job_details AS (
    SELECT *
    FROM dim_job_details
    WHERE job_id IN (
        SELECT job_id FROM enriched 
        WHERE job_standard_name = 'data analyst' )
--      WHERE job_standard_name = 'data engineer' )
--      WHERE job_standard_name = 'data scientist' )
    )

, enriched_details AS (
    SELECT *
            ,
CASE WHEN detail LIKE '%reporting%' THEN 'reporting' END AS reporting_counter,
CASE WHEN detail LIKE '%management%' THEN 'management' END AS management_counter,
CASE WHEN detail LIKE '%sql%' THEN 'sql' END AS sql_counter,
CASE WHEN detail LIKE '%insights%' THEN 'insights' END AS insights_counter,
CASE WHEN detail LIKE '%design%' THEN 'design' END AS design_counter,
CASE WHEN detail LIKE '%communication%' THEN 'communication' END AS communication_counter,
CASE WHEN detail LIKE '%modelling%' THEN 'modelling' END AS modelling_counter,
CASE WHEN detail LIKE '%excel%' THEN 'excel' END AS excel_counter,
CASE WHEN detail LIKE '%research%' THEN 'research' END AS research_counter,
CASE WHEN detail LIKE '%microsoft%' THEN 'microsoft' END AS microsoft_counter,
CASE WHEN detail LIKE '%dashboards%' THEN 'dashboards' END AS dashboards_counter,
CASE WHEN detail LIKE '%tableau%' THEN 'tableau' END AS tableau_counter,
CASE WHEN detail LIKE '%azure%' THEN 'azure' END AS azure_counter,
CASE WHEN detail LIKE '%visualisation%' THEN 'visualisation' END AS visualisation_counter,
CASE WHEN detail LIKE '%training%' THEN 'training' END AS training_counter,
CASE WHEN detail LIKE '%python%' THEN 'python' END AS python_counter,
CASE WHEN detail LIKE '%testing%' THEN 'testing' END AS testing_counter,
CASE WHEN detail LIKE '%cloud%' THEN 'cloud' END AS cloud_counter,
CASE WHEN detail LIKE '% r %' THEN 'r' END AS r_counter,
CASE WHEN detail LIKE '%etl%' THEN 'etl' END AS etl_counter,
CASE WHEN detail LIKE '%power%bi%' THEN 'powerbi' END AS powerbi_counter,
CASE WHEN detail LIKE '%statistics%' THEN 'statistics' END AS statistics_counter,
CASE WHEN detail LIKE '%aws%' THEN 'aws' END AS aws_counter,
CASE WHEN detail LIKE '%sas%' THEN 'sas' END AS sas_counter,
CASE WHEN detail LIKE '%presentation%' THEN 'presentation' END AS presentation_counter,
CASE WHEN detail LIKE '%ssis%' THEN 'ssis' END AS ssis_counter,
CASE WHEN detail LIKE '%quantitative%' THEN 'quantitative' END AS quantitative_counter,
CASE WHEN detail LIKE '%ssrs%' THEN 'ssrs' END AS ssrs_counter,
CASE WHEN detail LIKE '%oracle%' THEN 'oracle' END AS oracle_counter,
CASE WHEN detail LIKE '%crm%' THEN 'crm' END AS crm_counter,
CASE WHEN detail LIKE '%dax%' THEN 'dax' END AS dax_counter,
CASE WHEN detail LIKE '%ssas%' THEN 'ssas' END AS ssas_counter
FROM filtered_job_details),

da_job_count AS (
    SELECT job_id, reporting_counter AS skill, MIN(line_id) AS line_id FROM skill_counter WHERE reporting_counter is not null group by 1 UNION
SELECT job_id, management_counter, MIN(line_id) AS line_id FROM skill_counter WHERE management_counter is not null group by 1 UNION
SELECT job_id, sql_counter, MIN(line_id) AS line_id FROM skill_counter WHERE sql_counter is not null group by 1 UNION
SELECT job_id, insights_counter, MIN(line_id) AS line_id FROM skill_counter WHERE insights_counter is not null group by 1 UNION
SELECT job_id, design_counter, MIN(line_id) AS line_id FROM skill_counter WHERE design_counter is not null group by 1 UNION
SELECT job_id, communication_counter, MIN(line_id) AS line_id FROM skill_counter WHERE communication_counter is not null group by 1 UNION
SELECT job_id, modelling_counter, MIN(line_id) AS line_id FROM skill_counter WHERE modelling_counter is not null group by 1 UNION
SELECT job_id, excel_counter, MIN(line_id) AS line_id FROM skill_counter WHERE excel_counter is not null group by 1 UNION
SELECT job_id, research_counter, MIN(line_id) AS line_id FROM skill_counter WHERE research_counter is not null group by 1 UNION
SELECT job_id, microsoft_counter, MIN(line_id) AS line_id FROM skill_counter WHERE microsoft_counter is not null group by 1 UNION
SELECT job_id, dashboards_counter, MIN(line_id) AS line_id FROM skill_counter WHERE dashboards_counter is not null group by 1 UNION
SELECT job_id, tableau_counter, MIN(line_id) AS line_id FROM skill_counter WHERE tableau_counter is not null group by 1 UNION
SELECT job_id, azure_counter, MIN(line_id) AS line_id FROM skill_counter WHERE azure_counter is not null group by 1 UNION
SELECT job_id, visualisation_counter, MIN(line_id) AS line_id FROM skill_counter WHERE visualisation_counter is not null group by 1 UNION
SELECT job_id, training_counter, MIN(line_id) AS line_id FROM skill_counter WHERE training_counter is not null group by 1 UNION
SELECT job_id, python_counter, MIN(line_id) AS line_id FROM skill_counter WHERE python_counter is not null group by 1 UNION
SELECT job_id, testing_counter, MIN(line_id) AS line_id FROM skill_counter WHERE testing_counter is not null group by 1 UNION
SELECT job_id, cloud_counter, MIN(line_id) AS line_id FROM skill_counter WHERE cloud_counter is not null group by 1 UNION
SELECT job_id, r_counter, MIN(line_id) AS line_id FROM skill_counter WHERE r_counter is not null group by 1 UNION
SELECT job_id, etl_counter, MIN(line_id) AS line_id FROM skill_counter WHERE etl_counter is not null group by 1 UNION
SELECT job_id, powerbi_counter, MIN(line_id) AS line_id FROM skill_counter WHERE powerbi_counter is not null group by 1 UNION
SELECT job_id, statistics_counter, MIN(line_id) AS line_id FROM skill_counter WHERE statistics_counter is not null group by 1 UNION
SELECT job_id, aws_counter, MIN(line_id) AS line_id FROM skill_counter WHERE aws_counter is not null group by 1 UNION
SELECT job_id, sas_counter, MIN(line_id) AS line_id FROM skill_counter WHERE sas_counter is not null group by 1 UNION
SELECT job_id, presentation_counter, MIN(line_id) AS line_id FROM skill_counter WHERE presentation_counter is not null group by 1 UNION
SELECT job_id, ssis_counter, MIN(line_id) AS line_id FROM skill_counter WHERE ssis_counter is not null group by 1 UNION
SELECT job_id, quantitative_counter, MIN(line_id) AS line_id FROM skill_counter WHERE quantitative_counter is not null group by 1 UNION
SELECT job_id, ssrs_counter, MIN(line_id) AS line_id FROM skill_counter WHERE ssrs_counter is not null group by 1 UNION
SELECT job_id, oracle_counter, MIN(line_id) AS line_id FROM skill_counter WHERE oracle_counter is not null group by 1 UNION
SELECT job_id, crm_counter, MIN(line_id) AS line_id FROM skill_counter WHERE crm_counter is not null group by 1 UNION
SELECT job_id, dax_counter, MIN(line_id) AS line_id FROM skill_counter WHERE dax_counter is not null group by 1 UNION
SELECT job_id, ssas_counter, MIN(line_id) AS line_id FROM skill_counter WHERE ssas_counter is not null group by 1 )

select * from da_job_count
```

### Ranked Line in the Job Description of the Skills

We assume that people always would like to put the most important information at the beginninng. For example, the topper the description in the job advertisement means the more important the skill is. Then we analyze the average appeared line number of these important skills. In order to illustrate further, we can categorize the line numbers into three range: 1~3, 4~5 and 6+.

``` sql
WITH 
enriched AS (
    SELECT job_id, job_title
            , CASE 
            WHEN job_title LIKE '%data%analyst%'  THEN 'data analyst'  
            WHEN job_title LIKE '%bi%'  THEN 'data analyst'  
            WHEN job_title LIKE '%tableau%'  THEN 'data analyst'
            WHEN job_title like '%sql%' THEN 'data analyst'     
            WHEN job_title LIKE '%analytic%'  THEN 'data analyst' 
            WHEN job_title LIKE '%data%modeller%'  THEN 'data analyst' 
            WHEN job_title LIKE '%visualisation%'  THEN 'data analyst' 
            WHEN job_title LIKE '%business%intelligence%'  THEN 'data analyst'  
            WHEN job_title like '%insight%' THEN 'data analyst'
            WHEN job_title like '%reporting%' THEN 'data analyst'
            WHEN job_title like '%model%' THEN 'data analyst'

            WHEN job_title like '%data%engineer%' THEN 'data engineer'
            WHEN job_title like '%warehouse%' THEN 'data engineer'
            WHEN job_title like '%architect%' THEN 'data engineer'
            WHEN job_title like '%snowflake%' THEN 'data engineer'
            WHEN job_title like '%etl%' THEN 'data engineer'
            WHEN job_title like '%api%' THEN 'data engineer'
            WHEN job_title like '%cloud%' THEN 'data engineer'
            WHEN job_title like '%aws%' THEN 'data engineer'
            WHEN job_title like '%kafka%' THEN 'data engineer'
            WHEN job_title like '%pipeline%' THEN 'data engineer'
            WHEN job_title like '%migration%' THEN 'data engineer'

            WHEN job_title like '%scientist%' THEN 'data scientist'
            WHEN job_title like '%science%' THEN 'data scientist'
            WHEN job_title like '%machine%learning%' THEN 'data scientist'
            WHEN job_title like '% ai%' THEN 'data scientist'
            WHEN job_title like '%ai %' THEN 'data scientist'
            WHEN job_title like '%ml%' THEN 'data scientist'
            WHEN job_title like '%computational%statistics%' then 'data scientist'
            WHEN job_title like '%quantitative%' then 'data scientist'
            WHEN job_title like '%artificial%intelligence%' then 'data scientist'

            WHEN job_title like '%business%analyst%' THEN 'else'
            WHEN job_title like '%accountant%' THEN 'else'
            WHEN job_title like '%sales%' THEN 'else'
            WHEN job_title like '%product%' THEN 'else'
            WHEN job_title like '%dev%ops%' THEN 'else'
            WHEN job_title like '%graduate%' THEN 'else'
            WHEN job_title LIKE '%developer%' THEN 'else'
            WHEN job_title LIKE '%functional%' THEN 'else'
            WHEN job_title LIKE '%financial%' THEN 'else'
            WHEN job_title LIKE '%finance%' THEN 'else'
            WHEN job_title LIKE '%manager%' THEN 'else'
            WHEN job_title LIKE '%project%' THEN 'else'
            WHEN job_title LIKE '%program%' THEN 'else'

            END AS job_standard_name
    FROM dim_job_info
    ORDER BY 2 DESC
    )

, filtered_job_details AS (
    SELECT *
    FROM dim_job_details
    WHERE job_id IN (
        SELECT job_id FROM enriched 
        WHERE job_standard_name = 'data analyst' )
--      WHERE job_standard_name = 'data engineer' )
--      WHERE job_standard_name = 'data scientist' )
    )

, enriched_details AS (
    SELECT *
            ,
CASE WHEN detail LIKE '%reporting%' THEN 'reporting' END AS reporting_counter,
CASE WHEN detail LIKE '%management%' THEN 'management' END AS management_counter,
CASE WHEN detail LIKE '%sql%' THEN 'sql' END AS sql_counter,
CASE WHEN detail LIKE '%insights%' THEN 'insights' END AS insights_counter,
CASE WHEN detail LIKE '%design%' THEN 'design' END AS design_counter,
CASE WHEN detail LIKE '%communication%' THEN 'communication' END AS communication_counter,
CASE WHEN detail LIKE '%modelling%' THEN 'modelling' END AS modelling_counter,
CASE WHEN detail LIKE '%excel%' THEN 'excel' END AS excel_counter,
CASE WHEN detail LIKE '%research%' THEN 'research' END AS research_counter,
CASE WHEN detail LIKE '%microsoft%' THEN 'microsoft' END AS microsoft_counter,
CASE WHEN detail LIKE '%dashboards%' THEN 'dashboards' END AS dashboards_counter,
CASE WHEN detail LIKE '%tableau%' THEN 'tableau' END AS tableau_counter,
CASE WHEN detail LIKE '%azure%' THEN 'azure' END AS azure_counter,
CASE WHEN detail LIKE '%visualisation%' THEN 'visualisation' END AS visualisation_counter,
CASE WHEN detail LIKE '%training%' THEN 'training' END AS training_counter,
CASE WHEN detail LIKE '%python%' THEN 'python' END AS python_counter,
CASE WHEN detail LIKE '%testing%' THEN 'testing' END AS testing_counter,
CASE WHEN detail LIKE '%cloud%' THEN 'cloud' END AS cloud_counter,
CASE WHEN detail LIKE '% r %' THEN 'r' END AS r_counter,
CASE WHEN detail LIKE '%etl%' THEN 'etl' END AS etl_counter,
CASE WHEN detail LIKE '%power%bi%' THEN 'powerbi' END AS powerbi_counter,
CASE WHEN detail LIKE '%statistics%' THEN 'statistics' END AS statistics_counter,
CASE WHEN detail LIKE '%aws%' THEN 'aws' END AS aws_counter,
CASE WHEN detail LIKE '%sas%' THEN 'sas' END AS sas_counter,
CASE WHEN detail LIKE '%presentation%' THEN 'presentation' END AS presentation_counter,
CASE WHEN detail LIKE '%ssis%' THEN 'ssis' END AS ssis_counter,
CASE WHEN detail LIKE '%quantitative%' THEN 'quantitative' END AS quantitative_counter,
CASE WHEN detail LIKE '%ssrs%' THEN 'ssrs' END AS ssrs_counter,
CASE WHEN detail LIKE '%oracle%' THEN 'oracle' END AS oracle_counter,
CASE WHEN detail LIKE '%crm%' THEN 'crm' END AS crm_counter,
CASE WHEN detail LIKE '%dax%' THEN 'dax' END AS dax_counter,
CASE WHEN detail LIKE '%ssas%' THEN 'ssas' END AS ssas_counter
FROM filtered_job_details),

da_avg_line AS (
SELECT reporting_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE reporting_counter is not null UNION
SELECT management_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE management_counter is not null UNION
SELECT sql_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE sql_counter is not null UNION
SELECT insights_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE insights_counter is not null UNION
SELECT design_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE design_counter is not null UNION
SELECT communication_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE communication_counter is not null UNION
SELECT modelling_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE modelling_counter is not null UNION
SELECT excel_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE excel_counter is not null UNION
SELECT research_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE research_counter is not null UNION
SELECT microsoft_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE microsoft_counter is not null UNION
SELECT dashboards_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE dashboards_counter is not null UNION
SELECT tableau_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE tableau_counter is not null UNION
SELECT azure_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE azure_counter is not null UNION
SELECT visualisation_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE visualisation_counter is not null UNION
SELECT training_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE training_counter is not null UNION
SELECT python_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE python_counter is not null UNION
SELECT testing_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE testing_counter is not null UNION
SELECT cloud_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE cloud_counter is not null UNION
SELECT r_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE r_counter is not null UNION
SELECT etl_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE etl_counter is not null UNION
SELECT powerbi_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE powerbi_counter is not null UNION
SELECT statistics_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE statistics_counter is not null UNION
SELECT aws_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE aws_counter is not null UNION
SELECT sas_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE sas_counter is not null UNION
SELECT presentation_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE presentation_counter is not null UNION
SELECT ssis_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE ssis_counter is not null UNION
SELECT quantitative_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE quantitative_counter is not null UNION
SELECT ssrs_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE ssrs_counter is not null UNION
SELECT oracle_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE oracle_counter is not null UNION
SELECT crm_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE crm_counter is not null UNION
SELECT dax_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE dax_counter is not null UNION
SELECT ssas_counter AS skill, AVG(line_id) AS avg_line FROM skill_counter WHERE ssas_counter is not null)

SELECT *
    , CASE 
        WHEN avg_line <= 3 THEN '1~3'
        WHEN (avg_line > 3 AND avg_line <= 5) THEN '4~5'
        ELSE '6+' END AS line_id_category
FROM da_avg_line

```